# Data

In [ ]:
#!value --name exampledata
Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0

Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0

In [ ]:
#!value --name exampledata2
Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0

Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0

In [ ]:
#!value --name data
Register A: 25358015
Register B: 0
Register C: 0

Program: 2,4,1,1,7,5,0,3,4,7,1,6,5,5,3,0

Register A: 25358015
Register B: 0
Register C: 0

Program: 2,4,1,1,7,5,0,3,4,7,1,6,5,5,3,0

# Utilities

In [ ]:
void Print(object s) {
    Console.WriteLine(s);
}

public static string DebugPrint<T>(this IEnumerable<T> self) =>
        new StringBuilder("[")
            .AppendJoin(", ", self)
            .Append(']')
            .ToString();

### Imports

In [ ]:
using System;
using System.Collections;
using System.Text.RegularExpressions;
using System.Linq;
using System.Diagnostics;
using System.Numerics;

### Data Selector

In [ ]:
#!set --name fullData --value @value:data
#!set --name partialData --value @value:exampledata
#!set --name partialData2 --value @value:exampledata2

var rawdata = fullData;
var data = rawdata.ReplaceLineEndings("\n");

# Part 1

In [ ]:
enum Op {
    ADV = 0,
    BXL = 1,
    BST = 2,
    JNZ = 3,
    BXC = 4,
    OUT = 5,
    BDV = 6,
    CDV = 7
}

In [ ]:
Regex regARegex = new(@"^Register A: (\d+)$", RegexOptions.Multiline);
Regex regBRegex = new(@"^Register B: (\d+)$", RegexOptions.Multiline);
Regex regCRegex = new(@"^Register C: (\d+)$", RegexOptions.Multiline);
Regex programRegex = new(@"^Program: ((?:\d,)+\d)$", RegexOptions.Multiline);


BigInteger InitialA = BigInteger.Parse(regARegex.Match(data).Groups[1].Value);
BigInteger InitialB = BigInteger.Parse(regBRegex.Match(data).Groups[1].Value);
BigInteger InitialC = BigInteger.Parse(regCRegex.Match(data).Groups[1].Value);
List<short> Program = programRegex.Match(data).Groups[1].Value.Split(",").Select(s => short.Parse(s)).ToList();

In [ ]:
struct State {
    public BigInteger A;
    public BigInteger B;
    public BigInteger C;
    public int IP;
    public List<BigInteger> Output;

    public State(){
        IP = 0;
        Output = [];
    }
    public State(State prior) {
        A = prior.A;
        B = prior.B;
        C = prior.C;
        IP = prior.IP + 2;
        Output = [..prior.Output];
    }
}

State initialState = new() {
    A = InitialA,
    B = InitialB,
    C = InitialC,
};

In [ ]:
State state1 = new State() {
    A = 1,
    B = 2,
    C = 3,
};

// display(state1);
// display(new State(state1) {
//     A = 7
// });

In [ ]:
// Assume that we have already checked that the IP has not walked off the end!
State RunStep(List<short> Program, State curr) {
    BigInteger Combo(int ptr) =>
        ptr switch {
            0 => 0,
            1 => 1,
            2 => 2,
            3 => 3,
            4 => curr.A,
            5 => curr.B,
            6 => curr.C,
            _ => throw new InvalidProgramException()
        };

    // display(((Op)Program[curr.IP], Program[curr.IP + 1]));

    return ((Op)Program[curr.IP], Program[curr.IP + 1]) switch {
        (Op.ADV, short operand) => new State(curr) {
            A = curr.A / BigInteger.Pow(2, (int)Combo(operand))
        },
        (Op.BDV, short operand) => new State(curr) {
            B = curr.A / BigInteger.Pow(2, (int)Combo(operand))
        },
        (Op.CDV, short operand) => new State(curr) {
            C = curr.A / BigInteger.Pow(2, (int)Combo(operand))
        },
        (Op.BXL, short operand) => new State(curr) {
            B = curr.B ^ operand
        },
        (Op.BST, short operand) => new State(curr) {
            B = Combo(operand) % 8
        },
        (Op.JNZ, short operand) => curr.A == 0 ? new State(curr) : new State(curr) {
            IP = operand
        },
        (Op.BXC, _) => new State(curr) {
            B = curr.B ^ curr.C
        },
        (Op.OUT, short operand) => new State(curr) {
            Output = [..curr.Output, Combo(operand) % 8]
        }
    };
}

In [ ]:
string RunProgram(List<short> program, State initialState) {
    State state = initialState;
    int steps = 0;
    while(state.IP < program.Count) {
        // display((steps, state));
        state = RunStep(program, state);
        steps++;
    }
    return String.Join(',', state.Output);
}

RunProgram(Program, initialState)

2,7,2,5,1,2,7,3,7

# Part 2

In [ ]:
struct State {
    public BigInteger A;
    public BigInteger B;
    public BigInteger C;
    public int IP;
    public List<BigInteger> Output;

    public State(){
        IP = 0;
        Output = [];
    }
    public State(State prior) {
        A = prior.A;
        B = prior.B;
        C = prior.C;
        IP = prior.IP + 2;
        Output = [..prior.Output];
    }
    public string StringOutput() => String.Join(',', Output);
    public string ID() => $"{A}|{B}|{C}|{IP}";
}

State initialState = new() {
    A = InitialA,
    B = InitialB,
    C = InitialC,
};

In [ ]:
// Assume that we have already checked that the IP has not walked off the end!
State RunStep(List<short> Program, State curr) {
    BigInteger Combo(int ptr) =>
        ptr switch {
            0 => 0,
            1 => 1,
            2 => 2,
            3 => 3,
            4 => curr.A,
            5 => curr.B,
            6 => curr.C,
            _ => throw new InvalidProgramException()
        };

    // display(((Op)Program[curr.IP], Program[curr.IP + 1]));

    return ((Op)Program[curr.IP], Program[curr.IP + 1]) switch {
        (Op.ADV, short operand) => new State(curr) {
            A = curr.A / BigInteger.Pow(2, (int)Combo(operand))
        },
        (Op.BDV, short operand) => new State(curr) {
            B = curr.A / BigInteger.Pow(2, (int)Combo(operand))
        },
        (Op.CDV, short operand) => new State(curr) {
            C = curr.A / BigInteger.Pow(2, (int)Combo(operand))
        },
        (Op.BXL, short operand) => new State(curr) {
            B = curr.B ^ operand
        },
        (Op.BST, short operand) => new State(curr) {
            B = Combo(operand) % 8
        },
        (Op.JNZ, short operand) => curr.A == 0 ? new State(curr) : new State(curr) {
            IP = operand
        },
        (Op.BXC, _) => new State(curr) {
            B = curr.B ^ curr.C
        },
        (Op.OUT, short operand) => new State(curr) {
            Output = [..curr.Output, Combo(operand) % 8]
        }
    };
}

In [ ]:
public static string ToOctalString(this BigInteger bigint) {
    var bytes = bigint.ToByteArray();
    var idx = bytes.Length - 1;

    // Create a StringBuilder having appropriate capacity.
    var base8 = new StringBuilder(((bytes.Length / 3) + 1) * 8);

    // Calculate how many bytes are extra when byte array is split
    // into three-byte (24-bit) chunks.
    var extra = bytes.Length % 3;

    // If no bytes are extra, use three bytes for first chunk.
    if (extra == 0)
    {
      extra = 3;
    }

    // Convert first chunk (24-bits) to integer value.
    int int24 = 0;
    for (; extra != 0; extra--)
    {
      int24 <<= 8;
      int24 += bytes[idx--];
    }

    // Convert 24-bit integer to octal without adding leading zeros.
    var octal = Convert.ToString(int24, 8);

    // Ensure leading zero exists if value is positive.
    if (octal[0] != '0' && bigint.Sign == 1)
    {
      base8.Append('0');
    }

    // Append first converted chunk to StringBuilder.
    base8.Append(octal);

    // Convert remaining 24-bit chunks, adding leading zeros.
    for (; idx >= 0; idx -= 3)
    {
      int24 = (bytes[idx] << 16) + (bytes[idx - 1] << 8) + bytes[idx - 2];
      base8.Append(Convert.ToString(int24, 8).PadLeft(8, '0'));
    }

    return base8.ToString();
}

In [ ]:
HashSet<string> SeenStates = [];

List<BigInteger> RunProgram(List<short> program, State initialState, string targetOutput = "") {
    State state = initialState;
    int steps = 0;
    // BigInteger lastKnownA = 0;
    string lastKnownOutput = "";
    while(state.IP < program.Count) {
        // if (state.A != lastKnownA) {
        //     display(state.A.ToOctalString());
        //     lastKnownA = state.A;
        // }
        if (state.StringOutput() != lastKnownOutput) {
           lastKnownOutput = state.StringOutput();
        //    display((state.A.ToOctalString(), state.B.ToOctalString(), state.C.ToOctalString()));
        }
        // string id = state.ID();
        // if (SeenStates.Contains(id)) {
        //     return "SEEN";
        // }
        // SeenStates.Add(id);
        if (targetOutput.Length > 0 && !targetOutput.StartsWith(state.StringOutput())) {
            return null;
        }

        // display((steps, state));
        state = RunStep(program, state);
        steps++;
    }
    return state.Output;
}

RunProgram(Program, initialState)

index,value
0,2
1,7
2,2
3,5
4,1
5,2
6,7
7,3
8,7


In [ ]:
return; 
string target = String.Join(',', Program);
display(target);

string result = "";
State state = new State(initialState) {
    A = -1,
    IP = 0,
};

SeenStates.Clear();
Stopwatch timeout = Stopwatch.StartNew();

while (result != target && timeout.Elapsed < TimeSpan.FromSeconds(10)) {
    state = new State(state) {
        A = state.A + 1,
        IP = 0,
    };
    // HACK! This is not going to give valid output but it compiles
    result = RunProgram(Program, state, target).ToString();
    // if (loopTime.Elapsed > TimeSpan.FromMilliseconds(15)) {
    //     break;
    // }
    // loopTime.Restart();
}
display(state);


<null>

In [ ]:
void VisualizeProgram(List<short> program) {
    for(int i = 0; i < program.Count; i+= 2) {
        display(((Op)program[i], program[i+1]));
    }
}
// VisualizeProgram(Program);

In [ ]:
// display(RunProgram(Program, new State() { A = 0 }));
// display(RunProgram(Program, new State() { A = 1 }));

// for (int i = 0b001_000_000_000; i < Math.Pow(8, 4); i = i + 0b001_000) {
//     Print($"{Convert.ToString(i, 8)} {RunProgram(Program, new State() {A = i})}");
// }

In [ ]:
BigInteger minValue = BigInteger.Pow(8, Program.Count - 1);
BigInteger maxValue = BigInteger.Pow(8, Program.Count) - 1;

display(minValue);
display(minValue.ToOctalString());
display(maxValue);
display(maxValue.ToOctalString());

display(maxValue - minValue);

35184372088832

01000000000000000

281474976710655

07777777777777777

246290604621823

In [ ]:
return;

int[] octalDigits = new int[Program.Count];
for (int exponent = Program.Count - 1; exponent >= 0; exponent--) {
    for (int digit = 0; digit < 8; digit ++) {
        octalDigits[exponent] = digit;
        BigInteger testValue = 0; 
        for (int i = 0; i < Program.Count; i++) {
            testValue += BigInteger.Pow(8, i) * octalDigits[i];
        }
        if (testValue == 0) continue;

        var result = RunProgram(Program, new State() {A = testValue});

        // Print(Program.DebugPrint());
        // Print(result.DebugPrint());

        if (result[Program.Count - 1 - exponent] == Program[Program.Count - 1 - exponent]) {
            Print($"Locking {digit} at {exponent}");
            break;
        }
        if (digit == 7) {
            if (exponent < Program.Count -2) {
                exponent++;
                digit = octalDigits[exponent];
            }
        }
    }

    // display(octalDigits.DebugPrint());
}

BigInteger value = 0; 
for (int i = 0; i < Program.Count; i++) {
    value += BigInteger.Pow(8, i) * octalDigits[i];
}
display(value);
RunProgram(Program, new State() {A = value}).DebugPrint()

<null>

In [ ]:
BigInteger ValueFromOctalDigits(IEnumerable<int> octalDigits) {
    BigInteger result = 0;
    var digits = octalDigits.ToArray<int>();
    for (int i = 0; i < digits.Length; i++) {
        result += BigInteger.Pow(8, i) * digits[i];
    }
    return result;
}

State AState(BigInteger a) => new State() {A = a};

List<int> AlignBelowExponent(int exponent = 0, List<int> known = null) {
    // Print($"Called with {exponent} {known?.DebugPrint()}");
    if (exponent < 0) return known;
    int[] knownDigits = known?.ToArray() ?? [];
    for (int digit = 0; digit < 8; digit ++) {
        List<int> candidate = [..knownDigits, digit];
        while (candidate.Count < Program.Count) {
            candidate.Add(0);
        }
        candidate.Reverse();
        BigInteger value = ValueFromOctalDigits(candidate);
        if (value == 0) continue;

        var output = RunProgram(Program, AState(value));
        if (output.Count == Program.Count) {
            int posToCheck = exponent;
            if (output[posToCheck] == Program[posToCheck]) {
                Print(value);
                Print($"P {Program.DebugPrint()}\nO {output.DebugPrint()}\n");
                

                var result = AlignBelowExponent(exponent - 1, [..knownDigits, digit]);
                if (result is not null) {
                    return result;
                }
            }
        }
    }
    return null;
}

var result = AlignBelowExponent(Program.Count - 1);

246290604621824
P [2, 4, 1, 1, 7, 5, 0, 3, 4, 7, 1, 6, 5, 5, 3, 0]
O [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 3, 0]

246290604621824
P [2, 4, 1, 1, 7, 5, 0, 3, 4, 7, 1, 6, 5, 5, 3, 0]
O [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 3, 0]

247390116249600
P [2, 4, 1, 1, 7, 5, 0, 3, 4, 7, 1, 6, 5, 5, 3, 0]
O [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 5, 3, 0]

247802433110016
P [2, 4, 1, 1, 7, 5, 0, 3, 4, 7, 1, 6, 5, 5, 3, 0]
O [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 5, 5, 3, 0]

247836792848384
P [2, 4, 1, 1, 7, 5, 0, 3, 4, 7, 1, 6, 5, 5, 3, 0]
O [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 6, 5, 5, 3, 0]

247838940332032
P [2, 4, 1, 1, 7, 5, 0, 3, 4, 7, 1, 6, 5, 5, 3, 0]
O [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 6, 5, 5, 3, 0]

247838940332032
P [2, 4, 1, 1, 7, 5, 0, 3, 4, 7, 1, 6, 5, 5, 3, 0]
O [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 6, 5, 5, 3, 0]

247838990663680
P [2, 4, 1, 1, 7, 5, 0, 3, 4, 7, 1, 6, 5, 5, 3, 0]
O [7, 7, 7, 7, 7, 7, 7, 3, 4, 7, 1, 6, 5, 5, 3, 0]

247838990663680
P [2, 4, 1, 1, 7, 5, 0, 3, 4, 7,

In [ ]:
result.Reverse();
var answer = ValueFromOctalDigits(result);
Print(answer);

RunProgram(Program, AState(answer))

247839002892474


index,value
0,2
1,4
2,1
3,1
4,7
5,5
6,0
7,3
8,4
9,7
